In [4]:
import numpy as np
import os
from sklearn.metrics import confusion_matrix
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle           
import matplotlib.pyplot as plt             
import cv2                                 
import tensorflow as tf                
from tqdm import tqdm

In [5]:
class_names = ['mountain', 'street', 'glacier', 'buildings', 'sea', 'forest']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (112, 112)

In [6]:
def load_data():
    
    datasets = ['/kaggle/input/intel-image-classification/seg_train/seg_train', '/kaggle/input/intel-image-classification/seg_test/seg_test']
    output = []
    
    # Iterate through training and test sets
    for dataset in datasets:
        
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        # Iterate through each folder corresponding to a category
        for folder in os.listdir(dataset):
            label = class_names_label[folder]
            
            # Iterate through each image in our folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                
                # Get the path name of the image
                img_path = os.path.join(os.path.join(dataset, folder), file)
                
                # Open and resize the img
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE) 
                
                # Append the image and its corresponding label to the output
                images.append(image)
                labels.append(label)
                
        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')   
        
        output.append((images, labels))

    return output

In [7]:
(train_images, train_labels), (test_image, test_labels) = load_data()

  1%|          | 22/2191 [00:00<00:09, 219.46it/s]

Loading /kaggle/input/intel-image-classification/seg_train/seg_train


  7%|▋         | 32/437 [00:00<00:01, 315.27it/s]

Loading /kaggle/input/intel-image-classification/seg_test/seg_test


100%|██████████| 501/501 [00:01<00:00, 342.03it/s]


In [8]:
from tensorflow.keras.layers import *
from keras.models import Sequential

Classification_Model_Keras = Sequential([
        Conv2D(256,kernel_size=(3,3),activation='relu',input_shape=(112,112,3)),
        Conv2D(128,kernel_size=(3,3),activation='relu'),
        Conv2D(64,kernel_size=(3,3),activation='relu'),
        MaxPool2D(4,4),
        Conv2D(128,kernel_size=(3,3),activation='relu'),    
        Conv2D(64,kernel_size=(3,3),activation='relu'),    
        Conv2D(32,kernel_size=(3,3),activation='relu'),
        MaxPool2D(4,4),
        Flatten() ,    
        Dense(128,activation='relu') ,    
        Dense(64,activation='relu') ,    
        Dense(32,activation='relu') ,        
        Dropout(rate=0.5) ,            
        Dense(6,activation='softmax') ,    
        ])

In [10]:
Classification_Model_Keras.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 110, 110, 256)     7168      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 108, 108, 128)     295040    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 106, 106, 64)      73792     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 22, 22, 64)        73792     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 20, 32)        1

In [12]:
Classification_Model_Keras.compile(optimizer ='adam',
                                   loss='sparse_categorical_crossentropy',
                                   metrics=['accuracy'])

In [13]:
epochs = 40
KerasModel = Classification_Model_Keras.fit(train_images, train_labels, epochs=30,batch_size=64,verbose=1)

Epoch 1/10
220/220 [==============================] - 30s 137ms/step - loss: 1.7715 - accuracy: 0.2825
Epoch 2/10
220/220 [==============================] - 30s 136ms/step - loss: 1.1804 - accuracy: 0.5277
Epoch 3/10
220/220 [==============================] - 30s 135ms/step - loss: 1.0206 - accuracy: 0.5976
Epoch 4/10
220/220 [==============================] - 30s 136ms/step - loss: 0.9435 - accuracy: 0.6382
Epoch 5/10
220/220 [==============================] - 30s 135ms/step - loss: 0.8926 - accuracy: 0.6568
Epoch 6/10
220/220 [==============================] - 30s 135ms/step - loss: 0.8439 - accuracy: 0.6895
Epoch 7/10
220/220 [==============================] - 30s 135ms/step - loss: 0.8014 - accuracy: 0.7113
Epoch 8/10
220/220 [==============================] - 30s 136ms/step - loss: 0.7520 - accuracy: 0.7289
Epoch 9/10
220/220 [==============================] - 30s 135ms/step - loss: 0.7214 - accuracy: 0.7483
Epoch 10/10
220/220 [==============================] - 30s 136ms/step - l

In [14]:
test_loss, test_acc = Classification_Model_Keras.evaluate(test_image,  test_labels, verbose=1)
print('\n테스트 정확도:', test_acc)

94/94 [==============================] - 2s 24ms/step - loss: 0.6023 - accuracy: 0.7930

테스트 정확도: 0.7929999828338623
